# Financial Data Analysis Project

## Introduction:
In this notebook, I go through the end-to-end process of collecting, cleaning, exploring, analyzing, and visualizing data from Kaggle.com regarding financial metrics on S&P 500 companies. Analysis is done using Python and the following libraries/modules:
- numpy
- pandas
- matplotlib
- seaborn
- scikit-learn

### Goals / Objectives:
1. What relations exist between S&P 500 companies and key financial metrics?
2. How can information on these companies be used by investors to make financial decisions?
3. How does net income differ from by industry/sector?
    - What is the distribution of industries like for companies in the S&P 500?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import math

## Collecting the Data:
Step 1: Importing datasets and initial look into the data

The following datasets were found on Kaggle.com and were loaded in using pandas dataframes. The datasets contain quantitative information on key financial metrics including net income and market cap, in addition to categorical data such as industry, sub-industry, and headquarters location

In [ ]:
df1 = pd.read_csv('financial data sp500 companies.csv', encoding='windows-1252')
df2 = pd.read_csv('sp500-companies.csv', encoding='windows-1252')
df3 = pd.read_csv('constituents-financials_csv.csv', encoding='windows-1252')

print('df1 columns:\n', df1.columns, '\n')
print('df2 columns:\n', df2.columns, '\n')
print('df3 columns:\n', df3.columns, '\n')

In [ ]:
# Merging data into master dataframe
df_merge = df1.merge(df2, how='inner', on='Ticker')
df_merge = df_merge.merge(df3, how='inner', left_on='Ticker', right_on='Symbol')

pd.set_option('display.max_columns', None)
df_merge.head()

In [ ]:
# Looking at information on each column
df_merge.info()

In [ ]:
# Dimensions of dataframe
print('Number of rows and columns', df_merge.shape)

# checking for null/NaN values in each column
df_merge.isnull().sum()

## Data Cleaning and Preprocessing:
Step 2: Data preparation before analysis

Preparation includes removal of unnecessary columns, standardizing column headers, changing column datatypes, adding new columns, changing the format of numbers, dealing with null/NaN values, etc.

In [ ]:
# Removing unwanted columns
df_merge = df_merge.drop(columns=['Name_x','Name_y','Symbol','Sector','SEC Filings',
                                  '52 Week Low','52 Week High','Price','Date added',
                                  'Founded','Research Development', 'Unnamed: 0'],
                         axis=1)

# renaming columns for consistency in capitalization
df_merge = df_merge.rename(columns={'date':'Date',
                                    'firm':'Firm',
                                    'Ebit':'EBIT'})

# changing Date column into a datetime data type
df_merge['Date'] = pd.to_datetime(df_merge['Date'])

# Separating headquarter city and state into its own columns
df_merge[['Headquarter City', 'Headquarter State']] = df_merge['Headquarters Location'].str.split(',',1,expand=True)
df_merge = df_merge.drop(columns='Headquarters Location', axis=1)

# changing format of numbers
pd.options.display.float_format = '{:.2f}'.format

df_merge

In [ ]:
# grouping data by Ticker symbol and taking the average for numerical data
df_summary = df_merge.groupby('Ticker').mean()
pd.merge(df_summary, df2, on='Ticker')
df_summary = df_summary.merge(df2,on='Ticker')
df_summary[['Headquarter City', 'Headquarter State']] = df_summary['Headquarters Location'].str.split(',',1,expand=True)
df_summary = df_summary.drop(columns=['Headquarters Location','Sub-Industry','Date added','Founded','Price/Sales','Price/Book',
                                      'Total Other Income Expense Net','Net Income From Continuing Ops','Net Income Applicable To Common Shares',
                                      'Income Tax Expense','Interest Expense','Selling General Administrative','EBIT','EBITDA'],
                           axis=1)
df_summary

In [ ]:
# Taking a look at df_summary
companies = len(pd.unique(df_summary['Ticker']))
print('Number of companies in dataset:', companies,'\n')

print('Number of rows and columns:', df_summary.shape,'\n')

print('Column names:\n', df_summary.columns,'\n')

In [ ]:
# Descriptive statistics about the data
df_summary.describe()

In [ ]:
# checking for null/NaN values in each column
df_summary.isnull().sum()

In [ ]:
# seeing how many unique entries there are in each column
df_summary.nunique()

In [ ]:
# companies with highest market cap
df_summary.sort_values(by='Market Cap', ascending=False).head(10)

## Exploratory Data Analysis / Data Visualization:
Step 3: Analyzing data to find patterns, relationships, and trends

Visualizing the data using matplotlib and seaborn to obtain insights and provide strategic recommendations. Also using scikit-learn to model the data, specially with linear regression.

In [ ]:
# Bar plot depicting average net income by industry
plt.figure(figsize=(20,7))
sns.barplot(data = df_merge, x='Industry', y='Net Income')
plt.title('Average Net Income by Industry')
plt.xticks(rotation=30)
plt.ticklabel_format(style='plain', axis='y')
plt.show()

In [ ]:
# Count plot of companies for each industry
plt.figure(figsize=(15,5))
industry_count = sns.countplot(data=df_summary, x='Industry', order = df_summary['Industry'].value_counts(ascending=False).index)
plt.title('Number of Companies per Industry')
plt.xticks(rotation=30)

for i in industry_count.containers:
    industry_count.bar_label(i)
    
plt.show()

In [ ]:
# Visualizing the number of companies per industry in a pie chart
industries = df_summary['Industry'].unique()

industry_count = {}
for industry in industries:
    industry_count[industry] = 0

for _, row in df_summary.iterrows():
    industry_temp = row['Industry']
    industry_count[industry_temp] += 1
    
keys = list(industry_count.keys())
counts = list(industry_count.values())

plt.figure(figsize=(10,5))
plt.pie(counts, labels=keys, autopct='%.1f%%', explode = (0, 0, 0, 0, 0, 0.1, 0, 0 ,0.1 ,0 ,0))
plt.show()

In [ ]:
# Box plot showing dividend yield by industry
plt.figure(figsize=(15,5))
sns.boxplot(data = df_summary, x='Industry', y='Dividend Yield')
plt.title('Dividend Yield by Industry')
plt.xticks(rotation=30)
plt.show()

In [ ]:
# creating pair plot to see relation between various numeric fields
sns.pairplot(df_summary[['Net Income','Total Revenue','Total Operating Expenses','Dividend Yield','Market Cap']])

In [ ]:
# Scatter plot showing relation between net income and market cap
plt.figure(figsize=(10,5))
plt.scatter(data=df_summary, x='Market Cap', y='Net Income', alpha=0.5)
plt.title('Net Income by Market Cap')
plt.xlabel('Market Cap')
plt.ylabel('Net Income')
plt.ticklabel_format(style='plain')
plt.xticks(rotation=30)
plt.show()

In [ ]:
# Test train split for scikit-learn linear regression model
x_train, x_test, y_train, y_test = train_test_split(df_summary['Market Cap'], df_summary['Net Income'])

In [ ]:
# Test train split visualization
plt.figure(figsize=(10,5))
plt.scatter(x_train, y_train, label='Training Data', color='r', alpha=0.5)
plt.scatter(x_test, y_test, label='Testing Data', color='g', alpha=0.5)
plt.ticklabel_format(style='plain')
plt.xticks(rotation=30)
plt.legend()
plt.title('Test Train Split')
plt.xlabel('Market Cap')
plt.ylabel('Net Income')
plt.show()

In [ ]:
# Creating and training linear model
lr = LinearRegression()
lr.fit(x_train.values.reshape(-1,1), y_train.values)

# Using linear model to predict test data
prediction = lr.predict(x_test.values.reshape(-1,1))

# Plotting prediction line against actual test data
plt.figure(figsize=(10,5))
plt.plot(x_test, prediction, label='Linear Regression', color='b')
plt.scatter(x_test, y_test, label='Actual Test Data', color='g', alpha = 0.5)
plt.ticklabel_format(style='plain')
plt.xticks(rotation=30)
plt.xlabel('Market Cap')
plt.ylabel('Net Income')
plt.legend()
plt.show()

In [ ]:
# Predict Net Income of company with Market Cap of 300 billion
lr.predict(np.array([[300000000000]]))[0]

In [ ]:
# Score model
score = lr.score(x_test.values.reshape(-1,1), y_test.values)
print(f'The coefficient of determination (r^2): {score}\nThis indicates that {round(score,4)*100}% of the variation in Net Income is attributable to the company\'s Market Cap.' )

correlation = math.sqrt(score)
print(f'Correlation (r): {round(correlation,2)}' )

## Summary:
Step 4: Findings, reflections, challenges, etc.

### Goals / Objectives:
1. What relations exist between S&P 500 companies and key financial metrics?
2. How can information on these companies be used by investors to make financial decisions?
3. How does net income differ from by industry/sector?
    - What is the distribution of industries like for companies in the S&P 500?

### Challenges:


### Next Steps:

